[Reference](https://medium.com/geekculture/feature-engineering-for-equipment-failure-and-predictive-maintenance-problems-d72e6a3ca2bf)

In [1]:
!pip install plotly --upgrade
!pip install chart-studio --upgrade

     |████████████████████████████████| 27.7 MB 19.0 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
     |████████████████████████████████| 64 kB 1.8 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=2915bc1756ed1938dd4ebbf61310d327ecd1ffcbcf75a803790e4b3ba342d3ee
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [2]:
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly as plotly
import pandas as pd
import numpy as np

In [3]:
#Remove the data if you run this notebook more than once
!rm fe_equipment_failure_data_1.csv#import first half from github
!wget https://raw.githubusercontent.com/shadgriffin/feature_engineering_equipment_failure/main/fe_equipment_failure_data_1.csv# Convert csv to pandas dataframe
pd_data_1 = pd.read_csv("fe_equipment_failure_data_1.csv", sep=",", header=0)#Remove the data if you run this notebook more than once
!rm fe_equipment_failure_data_2.csv#Import the second half from github
!wget https://raw.githubusercontent.com/shadgriffin/feature_engineering_equipment_failure/main/fe_equipment_failure_data_2.csv# convert to pandas dataframe
pd_data_2 = pd.read_csv("fe_equipment_failure_data_2.csv", sep=",", header=0)#concatenate the two data files into one dataframe
pd_data=pd.concat([pd_data_1, pd_data_2])

rm: cannot remove 'fe_equipment_failure_data_1.csv#import': No such file or directory
rm: cannot remove 'first': No such file or directory
rm: cannot remove 'half': No such file or directory
rm: cannot remove 'from': No such file or directory
rm: cannot remove 'github': No such file or directory
--2022-03-30 12:12:57--  https://raw.githubusercontent.com/shadgriffin/feature_engineering_equipment_failure/main/fe_equipment_failure_data_1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3921508 (3.7M) [text/plain]
Saving to: ‘fe_equipment_failure_data_1.csv’

fe_equipment_failur 100%[===================>]   3.74M  --.-KB/s    in 0.07s   

2022-03-30 12:12:58 (54.5 MB/s) - ‘fe_equipment_failure_data_1.csv’ saved [3921508/3921508]

--2022-03-30 12:12:58--  http

In [4]:
pd_data.head()

,ID,DATE,MANUFACTURER,S15,EQUIPMENT_FAILURE
0,100001,12/2/14,Y,11.088000,0
1,100001,12/3/14,Y,8.877943,0
2,100001,12/4/14,Y,8.676444,0
3,100001,12/5/14,Y,9.988338,0
4,100001,12/6/14,Y,8.475264,0


In [5]:
pd_data.shape

(307751, 5)

In [6]:
xxxx = pd.DataFrame(pd_data.groupby(['ID']).agg(['count']))
xxxx.shape

(421, 4)

In [7]:
xxxx = pd.DataFrame(pd_data.groupby(['DATE']).agg(['count']))
xxxx.shape

(731, 4)

In [8]:
df_failure_thingy=pd_data
df_failure_thingy=df_failure_thingy.drop_duplicates(subset=['ID','DATE'])
df_failure_thingy.shape

(307751, 5)

In [9]:
pd_data.describe()

,ID,S15,EQUIPMENT_FAILURE
count,307751.000000,307751.000000,307751.000000
mean,100310.826603,14.585192,0.001368
std,177.574390,8.817056,0.036961
min,100001.000000,0.000000,0.000000
25%,100161.000000,7.694100,0.000000
50%,100311.000000,11.661600,0.000000
75%,100467.000000,22.560000,0.000000
max,100617.000000,59.040000,1.000000


In [10]:
xxxx = pd.DataFrame(pd_data.groupby(['MANUFACTURER'])['ID'].agg('count'))
xxxx

,ID
MANUFACTURER,
Q,27778
R,23392
S,38012
T,43129
U,31433
V,24854
W,32164
X,33626
Y,27778


In [12]:
df_dv = pd.get_dummies(pd_data['MANUFACTURER'])
pd_data = pd.concat([pd_data, df_dv], axis=1)
pd_data.head()

,ID,DATE,MANUFACTURER,S15,EQUIPMENT_FAILURE,Q,R,S,T,U,V,W,X,Y,Z
0,100001,12/2/14,Y,11.088000,0,0,0,0,0,0,0,0,0,1,0
1,100001,12/3/14,Y,8.877943,0,0,0,0,0,0,0,0,0,1,0
2,100001,12/4/14,Y,8.676444,0,0,0,0,0,0,0,0,0,1,0
3,100001,12/5/14,Y,9.988338,0,0,0,0,0,0,0,0,0,1,0
4,100001,12/6/14,Y,8.475264,0,0,0,0,0,0,0,0,0,1,0


In [15]:
#add 1 to S15 5o prevent because the natural log of 0 is undefined.
pd_data['LN_S15']=np.log((pd_data['S15']+1))
pd_data['INV_S15']=1/(pd_data['S15'])
pd_data['SQRT_S15']=np.sqrt((pd_data['S15'])+1)

In [18]:
d=pd_data[pd_data['ID']==100002]
x1=d['DATE']
y1=d['S15']
z1=d['INV_S15']
z2=d['LN_S15']
z3=d['SQRT_S15']
trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=z1,
    name='Inverse',
    yaxis='y2'
)
trace3 = go.Scatter(
    x=x1,
    y=z2,
    name='Natural Log',
    yaxis='y1'
)
trace4 = go.Scatter(
    x=x1,
    y=z3,
    name='Square Root',
    yaxis='y1'
)
data = [trace1, trace2,trace3, trace4]
layout = go.Layout(
    title='S15 and Static Transformations',
    yaxis=dict(title='S15'
    ),
        yaxis2=dict(
        title='Inverse',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes-lines')

In [19]:
#use two variables to create an interactive term
pd_data['S15_Y']=(pd_data['S15']*pd_data['Y'])
pd_data['S15_Z']=(pd_data['S15']*pd_data['Z'])

In [21]:
d=pd_data[pd_data['ID']==100002]
x1=d['DATE']
y1=d['S15']
z1=d['EQUIPMENT_FAILURE']
trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=z1,
    name='Failure',
    yaxis='y2'
)
data = [trace1, trace2]
layout = go.Layout(
    title='S15 Vs Failure',
    yaxis=dict(
        title='S15'
    ),
    yaxis2=dict(
        title='Failure',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes-lines')

In [23]:
d=pd_data[pd_data['ID']==100024]
x1=d['DATE']
y1=d['S15']
z1=d['EQUIPMENT_FAILURE']
trace1 = go.Scatter(
 x=x1,
 y=y1,
 name='S15'
)
trace2 = go.Scatter(
 x=x1,
 y=z1,
 name='Failure',
 yaxis='y2'
)
data = [trace1, trace2]
layout = go.Layout(
 title='S15 Vs Failure',
 yaxis=dict(
 title='S15'
 ),
 yaxis2=dict(
 title='Failure',
 titlefont=dict(
 color='rgb(148, 103, 189)'
 ),
 tickfont=dict(
 color='rgb(148, 103, 189)'
 ),
 overlaying='y',
 side='right'
 ),
 xaxis=dict(
 title='Date',
 titlefont=dict(
 family='Courier New, monospace',
 size=18,
 color='#7f7f7f'
 )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes-lines')

In [24]:
d=pd_data[pd_data['ID']==100131]
x1=d['DATE']
y1=d['S15']
z1=d['EQUIPMENT_FAILURE']
trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=z1,
    name='Failure',
    yaxis='y2'
)
data = [trace1, trace2]
layout = go.Layout(
    title='S15 Vs Failure',
    yaxis=dict(
        title='S15'
    ),
    yaxis2=dict(
        title='Failure',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes-lines')

In [25]:
#transform date to a datetime variable
pd_data['DATE'] = pd.to_datetime(pd_data['DATE'])

In [28]:
pd_data=pd_data.sort_values(by=['ID','DATE'], ascending=[True, True])

pd_data['flipper'] = np.where((pd_data.ID != pd_data.ID.shift(1)), 1, 0)
pd_data.head()

,ID,DATE,MANUFACTURER,S15,EQUIPMENT_FAILURE,Q,R,S,T,U,...,W,X,Y,Z,LN_S15,INV_S15,SQRT_S15,S15_Y,S15_Z,flipper
0,100001,2014-12-02,Y,11.088000,0,0,0,0,0,0,...,0,0,1,0,2.492213,0.090188,3.476780,11.088000,0.0,1
1,100001,2014-12-03,Y,8.877943,0,0,0,0,0,0,...,0,0,1,0,2.290304,0.112639,3.142920,8.877943,0.0,0
2,100001,2014-12-04,Y,8.676444,0,0,0,0,0,0,...,0,0,1,0,2.269694,0.115255,3.110698,8.676444,0.0,0
3,100001,2014-12-05,Y,9.988338,0,0,0,0,0,0,...,0,0,1,0,2.396835,0.100117,3.314866,9.988338,0.0,0
4,100001,2014-12-06,Y,8.475264,0,0,0,0,0,0,...,0,0,1,0,2.248685,0.117990,3.078192,8.475264,0.0,0


In [30]:
#define your feature window. This is the window by which we will aggregate our sensor values.
feature_window=14

In [31]:
dfx=pd_data
#Select the first record of each machine
starter=dfx[dfx['flipper'] == 1]
starter=starter[['DATE','ID']]

#rename date to start_date
starter=starter.rename(index=str, columns={"DATE": "START_DATE"})

#convert START_DATE to date
starter['START_DATE'] = pd.to_datetime(starter['START_DATE'])

#Merge START_DATE to the original data set
dfx=dfx.sort_values(by=['ID', 'DATE'], ascending=[True, True])
starter=starter.sort_values(by=['ID'], ascending=[True])
dfx =dfx.merge(starter, on=['ID'], how='left')

# calculate the number of days since the beginning of each well. 
dfx['C'] = dfx['DATE'] - dfx['START_DATE']
dfx['TIME_SINCE_START'] = dfx['C'] / np.timedelta64(1, 'D')
dfx=dfx.drop(columns=['C'])
dfx['too_soon'] = np.where((dfx.TIME_SINCE_START < feature_window) , 1, 0)

In [32]:
shaggy='S15'

In [33]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer=shaggy+'_'+qq+'_'+'M'
    dfx[namer] = np.where((dfx.too_soon == 0),(dfx[shaggy].rolling(min_periods=1, window=feature_window).mean()) , dfx[shaggy])

In [34]:
dfx.head()

,ID,DATE,MANUFACTURER,S15,EQUIPMENT_FAILURE,Q,R,S,T,U,...,S15_5_M,S15_6_M,S15_7_M,S15_8_M,S15_9_M,S15_10_M,S15_11_M,S15_12_M,S15_13_M,S15_14_M
0,100001,2014-12-02,Y,11.088000,0,0,0,0,0,0,...,11.088000,11.088000,11.088000,11.088000,11.088000,11.088000,11.088000,11.088000,11.088000,11.088000
1,100001,2014-12-03,Y,8.877943,0,0,0,0,0,0,...,8.877943,8.877943,8.877943,8.877943,8.877943,8.877943,8.877943,8.877943,8.877943,8.877943
2,100001,2014-12-04,Y,8.676444,0,0,0,0,0,0,...,8.676444,8.676444,8.676444,8.676444,8.676444,8.676444,8.676444,8.676444,8.676444,8.676444
3,100001,2014-12-05,Y,9.988338,0,0,0,0,0,0,...,9.988338,9.988338,9.988338,9.988338,9.988338,9.988338,9.988338,9.988338,9.988338,9.988338
4,100001,2014-12-06,Y,8.475264,0,0,0,0,0,0,...,8.475264,8.475264,8.475264,8.475264,8.475264,8.475264,8.475264,8.475264,8.475264,8.475264


In [35]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer=shaggy+'_'+qq+'_'+'X'
    dfx[namer] = np.where((dfx.too_soon == 0),(dfx[shaggy].rolling(min_periods=1, window=feature_window).max()) , dfx[shaggy])

In [37]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer=shaggy+'_'+qq+'_'+'N'
    dfx[namer] = np.where((dfx.too_soon == 0),(dfx[shaggy].rolling(min_periods=1, window=feature_window).min()) , dfx[shaggy])
    
dfx.head()

,ID,DATE,MANUFACTURER,S15,EQUIPMENT_FAILURE,Q,R,S,T,U,...,S15_5_N,S15_6_N,S15_7_N,S15_8_N,S15_9_N,S15_10_N,S15_11_N,S15_12_N,S15_13_N,S15_14_N
0,100001,2014-12-02,Y,11.088000,0,0,0,0,0,0,...,11.088000,11.088000,11.088000,11.088000,11.088000,11.088000,11.088000,11.088000,11.088000,11.088000
1,100001,2014-12-03,Y,8.877943,0,0,0,0,0,0,...,8.877943,8.877943,8.877943,8.877943,8.877943,8.877943,8.877943,8.877943,8.877943,8.877943
2,100001,2014-12-04,Y,8.676444,0,0,0,0,0,0,...,8.676444,8.676444,8.676444,8.676444,8.676444,8.676444,8.676444,8.676444,8.676444,8.676444
3,100001,2014-12-05,Y,9.988338,0,0,0,0,0,0,...,9.988338,9.988338,9.988338,9.988338,9.988338,9.988338,9.988338,9.988338,9.988338,9.988338
4,100001,2014-12-06,Y,8.475264,0,0,0,0,0,0,...,8.475264,8.475264,8.475264,8.475264,8.475264,8.475264,8.475264,8.475264,8.475264,8.475264


In [38]:
d=dfx[dfx['ID']==100131]
x1=d['DATE']
y1=d['S15']
y2=d['S15_12_M']
y3=d['S15_12_X']
y4=d['S15_12_N']
trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=y2,
    name='12 Day Mean'
)
trace3 = go.Scatter(
    x=x1,
    y=y3,
    name='12 Day Max'
)
trace4 = go.Scatter(
    x=x1,
    y=y4,
    name='12 Day Min'
)
data = [trace1,trace2,trace3,trace4]
layout = go.Layout(
    title='S15 and Summaries',
    yaxis=dict(
        title='S15'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes-lines')


In [39]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer='CHG'+'_'+shaggy+'_'+qq+'_'+'M'
    dfx[namer] = np.where((dfx[shaggy]> 0),(dfx[shaggy])/(dfx[shaggy+'_'+qq+'_'+'M']) , 0)

In [40]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer='CHG'+'_'+shaggy+'_'+qq+'_'+'N'
    dfx[namer] = np.where((dfx[shaggy]> 0),(dfx[shaggy])/(dfx[shaggy+'_'+qq+'_'+'N']) , 0)

In [41]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer='CHG'+'_'+shaggy+'_'+qq+'_'+'X'
    dfx[namer] = np.where((dfx[shaggy]> 0),(dfx[shaggy])/(dfx[shaggy+'_'+qq+'_'+'X']) , 0)

In [43]:
d=dfx[dfx['ID']==100131]
x1=d['DATE']
y1=d['S15']
y2=d['CHG_S15_12_M']
y3=d['CHG_S15_12_X']
y4=d['CHG_S15_12_N']
trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=y2,
    name='value/12 Day Mean',
    yaxis='y2'
)
trace3 = go.Scatter(
    x=x1,
    y=y3,
    name='value/12 Day Max',
    yaxis='y2'
)
trace4 = go.Scatter(
    x=x1,
    y=y4,
    name='value/12 Day Min',
    yaxis='y2'
)

data = [trace1,trace2,trace3,trace4]
layout = go.Layout(
    title='S15 and Summaries',
    yaxis=dict(
        title='S15'
    ),
    yaxis2=dict(
        title='Value over 12 Day Average',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes-lines')